In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.framework import graph_util
import cv2

begin=time.time()

# def img_preprocess(img):
    
#     return tf.image.convert_image_dtype(img, dtype=tf.float32)

def train_doc():
    
    filename_queue = tf.train.string_input_producer(['E:\\friedhelm\\face_detection_VGG\\face_train_224.tfrecords'],shuffle=True)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue) #返回文件名和文件

    features = tf.parse_single_example(serialized_example,
                                   features={
                                   'label':tf.FixedLenFeature([],tf.int64),
                                   'img':tf.FixedLenFeature([],tf.string),
                                   })
    img=tf.decode_raw(features['img'],tf.uint8)
    label=tf.cast(features['label'],tf.int32)
    img = tf.reshape(img, [224,224,3])   
#     img=img_preprocess(img)
    min_after_dequeue = 10000
    batch_size = 64
    capacity = min_after_dequeue + 10 * batch_size
    image_batch, label_batch = tf.train.shuffle_batch([img, label], 
                                                        batch_size=batch_size, 
                                                        capacity=capacity, 
                                                        min_after_dequeue=min_after_dequeue,
                                                        num_threads=7)  
    return image_batch,label_batch
    

def test_doc():
    
    filename_queue = tf.train.string_input_producer(['E:\\friedhelm\\face_detection_VGG\\face_test_224.tfrecords'],shuffle=True,seed=77)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue) #返回文件名和文件

    features = tf.parse_single_example(serialized_example,
                                   features={
                                   'label':tf.FixedLenFeature([],tf.int64),
                                   'img':tf.FixedLenFeature([],tf.string),
                                   })
    img=tf.decode_raw(features['img'],tf.uint8)
    label=tf.cast(features['label'],tf.int32)
    img = tf.reshape(img, [224,224,3])   
#     img=img_preprocess(img)
    min_after_dequeue = 1000
    batch_size = 16
    capacity = min_after_dequeue + 10 * batch_size
    image_batch, label_batch = tf.train.shuffle_batch([img, label], 
                                                        batch_size=batch_size, 
                                                        capacity=capacity, 
                                                        min_after_dequeue=min_after_dequeue,
                                                        num_threads=7)  
    return image_batch,label_batch    
    
    
def model(x,prob,is_training):
    
    with tf.variable_scope('conv1',reuse=tf.AUTO_REUSE):
        weight1=tf.get_variable('weight',[11,11,3,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias1=tf.get_variable('bias',[64],initializer=tf.constant_initializer(0.1))        
        conv1=tf.nn.conv2d(x,weight1,strides=[1,4,4,1],padding='SAME')
        he1=tf.nn.bias_add(conv1,bias1)
        bn1=tf.layers.batch_normalization(he1,training=is_training)
        relu1=tf.nn.relu(bn1)       

    with tf.variable_scope('pool1',reuse=tf.AUTO_REUSE):  
        pool1=tf.nn.max_pool(relu1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

    with tf.variable_scope('conv2',reuse=tf.AUTO_REUSE):
        weight2=tf.get_variable('weight',[5,5,64,192],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias2=tf.get_variable('bias',[192],initializer=tf.constant_initializer(0.1))        
        conv2=tf.nn.conv2d(pool1,weight2,strides=[1,1,1,1],padding='SAME')
        he2=tf.nn.bias_add(conv2,bias2)
        bn2=tf.layers.batch_normalization(he2,training=is_training)
        relu2=tf.nn.relu(bn2)     
    
    with tf.variable_scope('pool2',reuse=tf.AUTO_REUSE):  
        pool2=tf.nn.max_pool(relu2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
  
    with tf.variable_scope('conv3',reuse=tf.AUTO_REUSE):
        weight3=tf.get_variable('weight',[3,3,192,384],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias3=tf.get_variable('bias',[384],initializer=tf.constant_initializer(0.1))        
        conv3=tf.nn.conv2d(pool2,weight3,strides=[1,1,1,1],padding='SAME')
        he3=tf.nn.bias_add(conv3,bias3)
        mean3,vias3=tf.nn.moments(he3,0)
        bn3=tf.layers.batch_normalization(he3,training=is_training)
        relu3=tf.nn.relu(bn3)    
        
    with tf.variable_scope('conv4',reuse=tf.AUTO_REUSE):
        weight4=tf.get_variable('weight',[3,3,384,256],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias4=tf.get_variable('bias',[256],initializer=tf.constant_initializer(0.1))        
        conv4=tf.nn.conv2d(relu3,weight4,strides=[1,1,1,1],padding='SAME')
        he4=tf.nn.bias_add(conv4,bias4)
        bn4=tf.layers.batch_normalization(he4,training=is_training)
        relu4=tf.nn.relu(bn4)          
        
    with tf.variable_scope('conv5',reuse=tf.AUTO_REUSE):
        weight5=tf.get_variable('weight',[3,3,256,256],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias5=tf.get_variable('bias',[256],initializer=tf.constant_initializer(0.1))        
        conv5=tf.nn.conv2d(relu4,weight5,strides=[1,1,1,1],padding='SAME')
        he5=tf.nn.bias_add(conv5,bias5)
        bn5=tf.layers.batch_normalization(he5,training=is_training)
        relu5=tf.nn.relu(bn5)    

    with tf.variable_scope('pool3',reuse=tf.AUTO_REUSE):  
        pool3=tf.nn.max_pool(relu5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('fc1',reuse=tf.AUTO_REUSE):     
        weight6=tf.get_variable('weight',[6,6,256,1024],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias6=tf.get_variable('bias',[1024],initializer=tf.constant_initializer(0.1))        
        conv6=tf.nn.conv2d(pool3,weight6,strides=[1,1,1,1],padding='VALID')
        he6=tf.nn.bias_add(conv6,bias6)
        bn6=tf.layers.batch_normalization(he6,training=is_training)
        relu6=tf.nn.relu(bn6)     
        fc1=tf.nn.dropout(relu6,prob)

    with tf.variable_scope('fc2',reuse=tf.AUTO_REUSE):     
        weight7=tf.get_variable('weight',[1,1,1024,512],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias7=tf.get_variable('bias',[512],initializer=tf.constant_initializer(0.1))        
        conv7=tf.nn.conv2d(fc1,weight7,strides=[1,1,1,1],padding='VALID')
        he7=tf.nn.bias_add(conv7,bias7)
        bn7=tf.layers.batch_normalization(he7,training=is_training)
        relu7=tf.nn.relu(bn7)     
        fc2=tf.nn.dropout(relu7,prob)
        
    with tf.variable_scope('fc3',reuse=tf.AUTO_REUSE):     
        weight8=tf.get_variable('weight',[1,1,512,2],initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias8=tf.get_variable('bias',[2],initializer=tf.constant_initializer(0.1))        
        conv8=tf.nn.conv2d(fc2,weight8,strides=[1,1,1,1],padding='VALID')
        logit=tf.nn.bias_add(conv8,bias8,name='he')     
    return logit  
    
def train(x,y_,prob,is_training):
    
    y=model(x,prob,is_training)
    _y=tf.reshape(y,(-1,2))
    output=tf.nn.softmax(_y,name='logit') 

    with tf.name_scope('train_loss'):
#         loss_all=tf.add(-y_*tf.log(y_to_loss+1e-9),-(1-y_)*tf.log(1-y_to_loss+1e-9))
#         y_=tf.stop_gradient(y_)
        loss_all=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_,logits=_y)
        loss=tf.reduce_mean(loss_all,name='train_loss')
        tf.summary.scalar('loss',loss) 
        
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt=tf.train.AdamOptimizer(0.01).minimize(loss)    

    with tf.name_scope('test_accuracy'):
        test_accuracy=tf.reduce_mean(tf.cast(tf.equal(y_,tf.argmax(output,1)),tf.float32),name='test_accuracy')
        tf.summary.scalar('test_accuracy',test_accuracy) 


    saver=tf.train.Saver()
    merged=tf.summary.merge_all() 
    
    image_batch,label_batch=train_doc()
    image_batch1,label_batch1=test_doc()    
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
       
        writer_train=tf.summary.FileWriter('E:\\friedhelm\\face_detection_VGG\\train\\',sess.graph)
        writer_test=tf.summary.FileWriter('E:\\friedhelm\\face_detection_VGG\\test\\')
        
        for i in range(50001):
            img,label=sess.run([image_batch,label_batch])
            label=np.reshape(label,(64))    
            sess.run(opt,feed_dict={x:img,y_:label,prob:0.7,is_training:True})
            if(i%100==0):
                img1,label1=sess.run([image_batch1,label_batch1])
                label1=np.reshape(label1,(16))
                
                summary=sess.run(merged,feed_dict={x:img,y_:label,prob:0.7,is_training:True})
                writer_train.add_summary(summary,i) 
                
                summary,_=sess.run([merged,test_accuracy],feed_dict={x:img1,y_:label1,prob:1,is_training:False})
                writer_test.add_summary(summary,i) 
            if(i%1000==0):
                print('次数',i)    
                print('output',sess.run(output,feed_dict={x:img,y_:label,prob:0.7,is_training:True}))
                print(label)
                print('test_accuracy',sess.run(test_accuracy,feed_dict={x:img1,y_:label1,prob:1,is_training:False}))               
                print('loss',sess.run(loss,feed_dict={x:img,y_:label,prob:0.7,is_training:True}))               
                print('time',time.time()-begin)
                saver.save(sess,"E:\\friedhelm\\model\\model.ckpt")
    writer_train.close()
    writer_test.close()
    
def main():
    
    with tf.name_scope('input'):
        x=tf.placeholder(tf.float32,name='x')
        y_=tf.placeholder(tf.int64,name='y')
        prob=tf.placeholder(tf.float32,name='prob')
        is_training = tf.placeholder(tf.bool,name='is_train')    
    train(x,y_,prob,is_training)

if __name__=='__main__':
    main()
# tensorboard --logdir=C:\\Users\\312\\Desktop\\
# tensorboard --logdir=train:"E:\\friedhelm\\face_detection_VGG\\train\\",test:"E:\\friedhelm\\face_detection_VGG\\test\\"

次数 0
output [[6.7750480e-05 9.9993229e-01]
 [9.9999797e-01 2.0462403e-06]
 [2.2951723e-05 9.9997699e-01]
 [4.9499422e-09 1.0000000e+00]
 [9.9994922e-01 5.0803475e-05]
 [1.1086398e-04 9.9988914e-01]
 [5.1466596e-01 4.8533407e-01]
 [9.9999642e-01 3.5882247e-06]
 [9.9999404e-01 5.9674194e-06]
 [9.3230672e-02 9.0676928e-01]
 [9.9998748e-01 1.2560025e-05]
 [8.2999632e-02 9.1700041e-01]
 [9.9661094e-01 3.3890649e-03]
 [9.9992740e-01 7.2547627e-05]
 [9.9986100e-01 1.3897734e-04]
 [9.9999988e-01 1.5222261e-07]
 [4.0122334e-02 9.5987767e-01]
 [4.8708221e-01 5.1291776e-01]
 [4.4451354e-14 1.0000000e+00]
 [9.9997282e-01 2.7145197e-05]
 [9.9988317e-01 1.1681715e-04]
 [9.5574278e-01 4.4257265e-02]
 [9.9714822e-01 2.8518112e-03]
 [9.9945909e-01 5.4091983e-04]
 [9.9674749e-01 3.2525240e-03]
 [9.9999917e-01 8.0624591e-07]
 [9.3113953e-01 6.8860531e-02]
 [1.0000000e+00 5.7555891e-08]
 [9.9999797e-01 1.9780214e-06]
 [9.9927837e-01 7.2162534e-04]
 [9.9432087e-01 5.6791282e-03]
 [9.9984372e-01 1.5623809e-

次数 4000
output [[1.0915474e-02 9.8908454e-01]
 [5.5123287e-01 4.4876719e-01]
 [9.9974900e-01 2.5094836e-04]
 [9.9998295e-01 1.7095474e-05]
 [2.3097020e-02 9.7690296e-01]
 [2.1276684e-02 9.7872329e-01]
 [9.9976641e-01 2.3353830e-04]
 [9.9972826e-01 2.7167448e-04]
 [9.7203165e-01 2.7968358e-02]
 [8.5811466e-01 1.4188530e-01]
 [9.9564642e-01 4.3535372e-03]
 [8.3393738e-02 9.1660625e-01]
 [6.8852907e-01 3.1147093e-01]
 [3.9892361e-02 9.6010762e-01]
 [9.9861455e-01 1.3854111e-03]
 [9.9995136e-01 4.8680035e-05]
 [9.9708909e-01 2.9109586e-03]
 [5.2346241e-01 4.7653753e-01]
 [9.9981326e-01 1.8681551e-04]
 [4.1653737e-03 9.9583459e-01]
 [9.9354315e-01 6.4568883e-03]
 [9.9838901e-01 1.6110307e-03]
 [9.2502189e-01 7.4978136e-02]
 [9.9997056e-01 2.9398656e-05]
 [9.9776721e-01 2.2328452e-03]
 [4.1844090e-03 9.9581558e-01]
 [1.2153798e-02 9.8784620e-01]
 [9.6685112e-01 3.3148929e-02]
 [3.8413429e-03 9.9615866e-01]
 [9.4495720e-01 5.5042803e-02]
 [2.1165371e-02 9.7883463e-01]
 [7.3832471e-04 9.992616

次数 8000
output [[1.8756978e-01 8.1243014e-01]
 [9.7411555e-01 2.5884468e-02]
 [9.4922149e-01 5.0778516e-02]
 [9.9953914e-01 4.6083075e-04]
 [9.9944621e-01 5.5376638e-04]
 [9.9767905e-01 2.3210063e-03]
 [1.1162747e-02 9.8883724e-01]
 [9.9977905e-01 2.2099879e-04]
 [9.9989915e-01 1.0085989e-04]
 [1.3797040e-04 9.9986196e-01]
 [2.0799810e-01 7.9200190e-01]
 [9.5470464e-01 4.5295283e-02]
 [9.9827468e-01 1.7252931e-03]
 [9.9858570e-01 1.4142717e-03]
 [9.9777299e-01 2.2270032e-03]
 [4.2715939e-03 9.9572843e-01]
 [9.9999452e-01 5.4620623e-06]
 [9.9991798e-01 8.2045350e-05]
 [1.5648395e-02 9.8435163e-01]
 [2.2527461e-01 7.7472538e-01]
 [9.9999917e-01 7.8509754e-07]
 [9.9972898e-01 2.7096647e-04]
 [2.8226807e-04 9.9971765e-01]
 [9.7825468e-01 2.1745296e-02]
 [9.9604154e-01 3.9585009e-03]
 [1.2242166e-01 8.7757832e-01]
 [9.9994230e-01 5.7647252e-05]
 [1.7919834e-05 9.9998212e-01]
 [9.9796331e-01 2.0366984e-03]
 [9.9999940e-01 6.0409354e-07]
 [9.9998832e-01 1.1731963e-05]
 [9.5625204e-01 4.374799

次数 12000
output [[3.3875766e-01 6.6124231e-01]
 [3.5570088e-06 9.9999642e-01]
 [8.0887903e-04 9.9919111e-01]
 [9.9995816e-01 4.1839063e-05]
 [9.9889034e-01 1.1097178e-03]
 [9.3251866e-01 6.7481354e-02]
 [6.2117109e-04 9.9937880e-01]
 [9.9766648e-01 2.3335970e-03]
 [9.9992871e-01 7.1331488e-05]
 [9.9512476e-01 4.8753093e-03]
 [9.9999142e-01 8.5845259e-06]
 [9.9873644e-01 1.2635756e-03]
 [9.9997556e-01 2.4436515e-05]
 [9.9235690e-01 7.6431031e-03]
 [9.9990213e-01 9.7836048e-05]
 [9.9998844e-01 1.1591558e-05]
 [9.9998677e-01 1.3178711e-05]
 [1.2453065e-03 9.9875474e-01]
 [9.9925274e-01 7.4718404e-04]
 [1.2536270e-03 9.9874640e-01]
 [9.8345286e-01 1.6547114e-02]
 [1.6325543e-03 9.9836749e-01]
 [9.9999428e-01 5.7038810e-06]
 [9.9990392e-01 9.6123134e-05]
 [9.9999142e-01 8.5884731e-06]
 [9.9968362e-01 3.1640401e-04]
 [9.9413753e-01 5.8624246e-03]
 [9.9995351e-01 4.6485125e-05]
 [9.5761395e-01 4.2386085e-02]
 [9.9999547e-01 4.5631323e-06]
 [4.5943845e-05 9.9995410e-01]
 [9.9990034e-01 9.96765

次数 16000
output [[9.9910837e-01 8.9156366e-04]
 [9.9996805e-01 3.1954096e-05]
 [9.2783701e-01 7.2162978e-02]
 [9.9999237e-01 7.6574916e-06]
 [9.9999797e-01 2.0105260e-06]
 [9.4443746e-02 9.0555626e-01]
 [9.9989223e-01 1.0780966e-04]
 [9.9751854e-01 2.4815144e-03]
 [9.5893937e-01 4.1060593e-02]
 [2.5968960e-02 9.7403109e-01]
 [9.9889010e-01 1.1099132e-03]
 [9.9987304e-01 1.2694688e-04]
 [6.4189732e-03 9.9358106e-01]
 [9.9999857e-01 1.3785429e-06]
 [9.9895537e-01 1.0445560e-03]
 [9.9930108e-01 6.9897139e-04]
 [9.9998295e-01 1.7038326e-05]
 [9.9730289e-01 2.6971239e-03]
 [9.9895042e-01 1.0496277e-03]
 [9.9973589e-01 2.6412777e-04]
 [9.9519157e-01 4.8084771e-03]
 [9.9983537e-01 1.6465438e-04]
 [9.9997771e-01 2.2297329e-05]
 [9.8394960e-01 1.6050445e-02]
 [9.9995267e-01 4.7320620e-05]
 [9.9992490e-01 7.5054777e-05]
 [9.9998546e-01 1.4557216e-05]
 [9.9938369e-01 6.1627425e-04]
 [9.9903858e-01 9.6149417e-04]
 [9.9999905e-01 9.0146744e-07]
 [9.9319661e-01 6.8033864e-03]
 [9.9946946e-01 5.30608

...

次数 32000
output [[1.00000000e+00 3.30294947e-09]
 [9.99999881e-01 8.63179466e-08]
 [1.20513616e-02 9.87948656e-01]
 [9.99985695e-01 1.43193720e-05]
 [9.99995947e-01 3.99965484e-06]
 [9.99999881e-01 1.38759503e-07]
 [7.62617029e-03 9.92373765e-01]
 [1.00000000e+00 2.87957080e-09]
 [2.97553896e-04 9.99702394e-01]
 [1.21479249e-03 9.98785198e-01]
 [1.00000000e+00 1.45998161e-11]
 [1.00000000e+00 2.38648656e-10]
 [9.99995708e-01 4.29238571e-06]
 [1.00000000e+00 1.64172302e-08]
 [1.00000000e+00 1.31928735e-08]
 [9.99977589e-01 2.24322939e-05]
 [1.00000000e+00 4.67667219e-16]
 [9.90260422e-01 9.73956659e-03]
 [9.99999762e-01 2.76348061e-07]
 [2.36232635e-02 9.76376712e-01]
 [9.99989748e-01 1.03012471e-05]
 [9.99999642e-01 3.02808104e-07]
 [9.99998450e-01 1.58245109e-06]
 [9.99993682e-01 6.27953386e-06]
 [9.97241020e-01 2.75893905e-03]
 [9.99999881e-01 1.10509347e-07]
 [1.00000000e+00 4.26900348e-10]
 [9.99999881e-01 1.68403503e-07]
 [1.00000000e+00 4.96867125e-09]
 [9.99999881e-01 6.73834037

次数 36000
output [[3.7300205e-03 9.9626994e-01]
 [2.9895850e-07 9.9999964e-01]
 [1.0000000e+00 2.6629376e-10]
 [1.0000000e+00 1.0622950e-09]
 [1.0000000e+00 2.7648914e-09]
 [2.5892166e-02 9.7410786e-01]
 [9.9999213e-01 7.8955181e-06]
 [1.0000000e+00 2.8072538e-09]
 [1.7347145e-01 8.2652861e-01]
 [1.0000000e+00 2.9646170e-08]
 [1.0000000e+00 2.1749678e-11]
 [1.0000000e+00 6.8553142e-11]
 [1.8308601e-04 9.9981695e-01]
 [1.0000000e+00 6.1788595e-13]
 [1.0000000e+00 2.2932238e-10]
 [1.8906636e-02 9.8109335e-01]
 [1.0000000e+00 1.3190455e-16]
 [1.0000000e+00 2.1329273e-16]
 [9.9999988e-01 9.8263179e-08]
 [1.0000000e+00 4.5484987e-15]
 [1.0000000e+00 2.1500686e-10]
 [4.6578415e-02 9.5342153e-01]
 [9.9999917e-01 8.6425496e-07]
 [1.0000000e+00 7.8907574e-09]
 [9.9999833e-01 1.6398913e-06]
 [1.0000000e+00 2.0173931e-13]
 [1.0000000e+00 6.9516346e-14]
 [7.2778419e-02 9.2722160e-01]
 [1.0000000e+00 1.8666190e-14]
 [1.0000000e+00 8.1333135e-11]
 [2.3564771e-05 9.9997640e-01]
 [1.3107447e-02 9.86892

次数 40000
output [[5.49376309e-01 4.50623721e-01]
 [1.00000000e+00 2.07087748e-18]
 [9.99996424e-01 3.51728886e-06]
 [1.00000000e+00 2.91607636e-17]
 [9.99866724e-01 1.33215464e-04]
 [6.32576237e-04 9.99367416e-01]
 [1.00000000e+00 2.03728995e-11]
 [1.00000000e+00 2.26897943e-19]
 [9.99984741e-01 1.52561697e-05]
 [6.92222118e-01 3.07777882e-01]
 [4.39896621e-03 9.95601058e-01]
 [1.00000000e+00 5.13430271e-12]
 [9.93985236e-01 6.01476664e-03]
 [1.00000000e+00 2.06160344e-15]
 [9.99999046e-01 9.79102197e-07]
 [1.89373046e-02 9.81062710e-01]
 [1.00000000e+00 6.55898296e-13]
 [1.00000000e+00 8.18852017e-12]
 [1.00000000e+00 1.89507860e-13]
 [1.26684404e-06 9.99998689e-01]
 [9.99725163e-01 2.74819409e-04]
 [1.48172537e-02 9.85182703e-01]
 [1.13705923e-04 9.99886274e-01]
 [9.27448049e-02 9.07255113e-01]
 [1.00000000e+00 1.25455570e-12]
 [2.63906969e-03 9.97360885e-01]
 [1.00000000e+00 4.53424106e-15]
 [2.12829420e-03 9.97871637e-01]
 [1.00000000e+00 4.93738490e-08]
 [1.00000000e+00 5.02716357

次数 44000
output [[1.00000000e+00 3.79685026e-14]
 [1.00000000e+00 5.05779363e-10]
 [1.00000000e+00 4.47662574e-10]
 [2.79475772e-03 9.97205198e-01]
 [2.08304518e-05 9.99979138e-01]
 [1.75674483e-02 9.82432485e-01]
 [1.00000000e+00 2.11307025e-08]
 [1.00000000e+00 3.43933215e-11]
 [9.99998569e-01 1.39831150e-06]
 [1.00000000e+00 1.00320534e-08]
 [1.00000000e+00 6.68041844e-09]
 [1.00000000e+00 3.71967207e-10]
 [9.99999881e-01 1.00142969e-07]
 [1.00000000e+00 6.88625419e-15]
 [1.00000000e+00 1.77707779e-11]
 [1.00000000e+00 3.56126406e-10]
 [9.99992967e-01 7.07899426e-06]
 [1.00000000e+00 1.32889297e-10]
 [1.00000000e+00 5.60619780e-12]
 [9.99999762e-01 2.04270592e-07]
 [6.28653467e-02 9.37134624e-01]
 [1.00000000e+00 1.01273000e-11]
 [1.00000000e+00 1.32853242e-11]
 [1.00000000e+00 3.12254146e-11]
 [9.99998212e-01 1.84656108e-06]
 [1.00000000e+00 1.18210091e-08]
 [1.55277690e-03 9.98447239e-01]
 [9.99999285e-01 6.67569509e-07]
 [9.99998689e-01 1.29168745e-06]
 [1.35244380e-04 9.99864697

次数 48000
output [[9.9988830e-01 1.1173457e-04]
 [9.9996269e-01 3.7293386e-05]
 [9.9981052e-01 1.8956591e-04]
 [9.9428815e-01 5.7118372e-03]
 [9.9999547e-01 4.5436950e-06]
 [9.9999976e-01 2.3159301e-07]
 [9.9998903e-01 1.0979760e-05]
 [1.0000000e+00 9.2579361e-11]
 [9.9685645e-01 3.1434917e-03]
 [9.9994504e-01 5.4972144e-05]
 [1.0000000e+00 5.9211452e-08]
 [9.9249023e-01 7.5096991e-03]
 [9.9999976e-01 2.5715519e-07]
 [9.9960500e-01 3.9503895e-04]
 [1.0000000e+00 1.4132347e-10]
 [9.9509311e-01 4.9068979e-03]
 [1.2697417e-01 8.7302589e-01]
 [9.4068175e-10 1.0000000e+00]
 [9.9518234e-01 4.8176241e-03]
 [9.1891281e-02 9.0810871e-01]
 [9.9999964e-01 3.6061692e-07]
 [9.9973351e-01 2.6650049e-04]
 [9.9999750e-01 2.4548783e-06]
 [9.9999988e-01 1.4962227e-07]
 [9.9999905e-01 9.2989950e-07]
 [9.9996078e-01 3.9161638e-05]
 [9.9776316e-01 2.2368813e-03]
 [9.9985719e-01 1.4274211e-04]
 [1.0000000e+00 2.7011002e-09]
 [1.0000000e+00 3.5759236e-11]
 [9.9984932e-01 1.5065962e-04]
 [9.9999213e-01 7.91550